In [2]:
import cv2
from ultralytics import YOLO

# Cargar el modelo YOLO preentrenado
model = YOLO('yolov5s.pt')  # Puedes elegir otros modelos como yolov5m.pt, yolov5l.pt, etc.
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
# Capturar video de la cámara
cap = cv2.VideoCapture(0)

fatigado = 0
patrones_fatigado = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Procesar el frame con YOLO
    results = model(frame)

    # Obtener las detecciones
    detections = results[0].boxes  # Aquí accedemos a las cajas

    # Procesar las detecciones
    for box in detections:
        # La estructura puede variar, pero generalmente incluye las siguientes propiedades
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas de la caja (x1, y1, x2, y2)
        conf = box.conf[0]  # Confianza de la detección
        cls = box.cls[0]    # Clase de la detección

        if conf > 0.5:# and cls == 0:  # Filtrar por confianza
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Dibuja un rectángulo alrededor del objeto

            # Opcionalmente, puedes agregar texto con la clase y la confianza
            cv2.putText(frame, f'Clase: {int(cls)} Conf: {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            # Extraer la región de interés (ROI) para los ojos
            roi_gray = cv2.cvtColor(frame[int(y1):int(y1)+int(y2), int(x1):int(x1)+int(x2)], cv2.COLOR_BGR2GRAY)
            eyes = eye_cascade.detectMultiScale(roi_gray)

            for (ex, ey, ew, eh) in eyes:
                    # Dibujar los ojos detectados
                    cv2.rectangle(frame, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2)
                    

            # Verificar el estado de los ojos
            if len(eyes) == 0:
                fatigado += 1    
            else:
                fatigado = 0
            if fatigado > 5:
                patrones_fatigado += 1
                cv2.putText(frame, "Fatigado", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "Despierto", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Detección de Fatiga', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [ ]:
#movil
cls = 67

In [12]:
results[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9275])
data: tensor([[165.5900,  89.8396, 632.9856, 479.3329,   0.9275,   0.0000]])
id: None
is_track: False
orig_shape: (480, 640)
shape: torch.Size([1, 6])
xywh: tensor([[399.2878, 284.5862, 467.3956, 389.4933]])
xywhn: tensor([[0.6239, 0.5929, 0.7303, 0.8114]])
xyxy: tensor([[165.5900,  89.8396, 632.9856, 479.3329]])
xyxyn: tensor([[0.2587, 0.1872, 0.9890, 0.9986]])

In [8]:
import cv2
import numpy as np
from ultralytics import YOLO

# Cargar el modelo YOLO preentrenado
model = YOLO('yolov5s.pt')

# Inicializar la captura de video
cap = cv2.VideoCapture(0)

# Umbral para detectar si el conductor está distraído
DISTRACTION_THRESHOLD = 30  # grados

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Procesar el frame con YOLO
    results = model(frame)
    # Obtener las detecciones
    detections = results[0].boxes  # Aquí accedemos a las cajas

    # Procesar las detecciones
    for box in detections:
        # La estructura puede variar, pero generalmente incluye las siguientes propiedades
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas de la caja (x1, y1, x2, y2)
        conf = box.conf[0]  # Confianza de la detección
        cls = box.cls[0]    # Clase de la detección
        
        if cls == 0 and conf > 0.5:  # Clase 0 corresponde a "persona"
            # Dibuja un rectángulo alrededor del rostro
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

            # Calcular el centro del rostro
            face_center_x = (x1 + x2) // 2
            face_center_y = (y1 + y2) // 2
            
            # Aquí puedes agregar lógica para detectar los ojos
            # Por ejemplo, usa Haar Cascades o un modelo de detección para los ojos.
            eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
            roi_gray = cv2.cvtColor(frame[int(y1):int(y2), int(x1):int(x2)], cv2.COLOR_BGR2GRAY)
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)

            if len(eyes) >= 2:  # Necesitamos detectar al menos dos ojos
                # Calcular la dirección de la mirada en función de los ojos detectados
                for (ex, ey, ew, eh) in eyes:
                    # Dibujar los ojos detectados
                    cv2.rectangle(frame, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2)
                    
                    # Supongamos que la dirección de la carretera es vertical en el centro del frame
                    # Definimos un vector de dirección de la mirada
                    gaze_vector = (face_center_x - ex - ew // 2, face_center_y - ey - eh // 2)
                    
                    # Aquí se puede definir la lógica para calcular el ángulo entre el vector de mirada y la carretera.
                    # Esto requiere conocer la orientación de la carretera.
                    # En este ejemplo, vamos a simular esto.
                    
                    road_direction = (0, 1)  # Por ejemplo, hacia abajo
                    angle = np.arctan2(gaze_vector[1], gaze_vector[0]) - np.arctan2(road_direction[1], road_direction[0])
                    angle = np.degrees(angle)

                    # Comprobar si el ángulo excede el umbral de distracción
                    if abs(angle) > DISTRACTION_THRESHOLD:
                        cv2.putText(frame, "Distraccion", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    else:
                        cv2.putText(frame, "Atento", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Detección de Distracción', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


0: 480x640 1 person, 94.9ms
Speed: 3.8ms preprocess, 94.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 163.9ms
Speed: 5.2ms preprocess, 163.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 kite, 129.7ms
Speed: 2.4ms preprocess, 129.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 kite, 123.9ms
Speed: 2.0ms preprocess, 123.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 120.2ms
Speed: 4.2ms preprocess, 120.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



C:\Users\Lidia Mayor\AppData\Local\Temp\ipykernel_32932\2625659710.py:60: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  angle = np.arctan2(gaze_vector[1], gaze_vector[0]) - np.arctan2(road_direction[1], road_direction[0])
C:\Users\Lidia Mayor\AppData\Local\Temp\ipykernel_32932\2625659710.py:61: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  angle = np.degrees(angle)


0: 480x640 1 person, 124.3ms
Speed: 0.5ms preprocess, 124.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 kite, 121.8ms
Speed: 1.5ms preprocess, 121.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 110.5ms
Speed: 1.5ms preprocess, 110.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 kite, 105.4ms
Speed: 1.0ms preprocess, 105.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 98.1ms
Speed: 1.5ms preprocess, 98.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 94.3ms
Speed: 1.1ms preprocess, 94.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 96.4ms
Speed: 2.2ms preprocess, 96.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 kite, 93.1ms
Speed: 1.0ms preprocess, 93.1ms inference, 1.0ms postproce